In [ ]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import qiskit
from qiskit import IBMQ

In [ ]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-sherbrooke')
sydney = provider.get_backend('ibmq_sydney')

## Generate datetime list

In [ ]:
date_queries = [datetime.now() - i*timedelta(days=1) for i in range (30)]
syd_conf = sydney.configuration()

In [ ]:
single_gates = ['id', 'sx', 'x']
errors = ['T1', 'T2', 'readout_error']
double_gates = ['cx']

In [ ]:
df = pd.DataFrame(columns=['qubit', 'date', 'T1', 'T2', 'readout'] + single_gates)
for day in date_queries:
    prop = sydney.properties(datetime=day)
    for qubit in range(syd_conf.num_qubits):
        entry = {'qubit':qubit, 'date': day}
        for gate in single_gates:
            entry[gate] = prop.gate_error(gate, qubit)
        entry['T1'] = prop.t1(qubit)
        entry['T2'] = prop.t2(qubit)
        entry['readout'] = prop.readout_error(qubit)
        df = df.append(entry, ignore_index=True)
df = df.astype({'qubit': int, 'date':int})

In [ ]:
fig, axs = plt.subplots(4, figsize = (20,20))
sns.boxplot(ax=axs[0], x="qubit", y="x", data=df)
sns.boxplot(ax=axs[1], x="qubit", y="T1", data=df)
sns.boxplot(ax=axs[2], x="qubit", y="T2", data=df)
sns.boxplot(ax=axs[3], x="qubit", y="readout", data=df)

axs[0].set_ylabel('Single-qubit Pauli-X error rate')
axs[1].set_ylabel('T1 ($\mu$s)')
axs[2].set_ylabel('T2 ($\mu$s)')
axs[3].set_ylabel('Readout assignment error')

fig.tight_layout()
plt.savefig('error_distribution_sydney')

In [ ]:
df_cx = pd.DataFrame(columns=['couple', 'date', 'cx'])
for day in date_queries:
    prop = sydney.properties(datetime=day)
    for couple in syd_conf.coupling_map:
        entry = {'couple': couple, 'date': day, 'cx': prop.gate_error("cx", couple)}
        df_cx = df_cx.append(entry, ignore_index=True)
df_cx = df_cx.astype({'couple': str})

In [ ]:
fig, ax = plt.subplots(figsize = (20,10))
sns.boxplot(ax=ax, x="couple", y="cx", data=df_cx)
ax.set_ylabel('CNOT error rate')
ax.set_ylim(0, 0.065)

text = f'({[25,26]} : {0.152071}) was croped from the plot\nfor a better readability (occured on 2021-03-14).'

props = dict(boxstyle='round', facecolor='wheat', alpha=0.2)
ax.text(0.73, 0.95, text, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)

fig.tight_layout()
plt.xticks(rotation=45)
plt.savefig('cx_error_distribution_sydney')